In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from scipy.stats import loguniform, uniform, randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

c:\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.6) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


Import data

In [2]:
X_train = pd.read_excel("data/model_inputs/X_train.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
X_test = pd.read_excel("data/model_inputs/X_test.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_train = pd.read_excel("data/model_inputs/y_train.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')
y_test = pd.read_excel("data/model_inputs/y_test.xlsx").rename(columns={'Unnamed: 0':'date'}).set_index('date')

Tuning

In [3]:
def random_oversampler(X_train, y_train):
    oversample = RandomOverSampler(sampling_strategy='not majority')
    X_over, y_over = oversample.fit_resample(X_train, y_train)
    return X_over, y_over

logreg_distribution = {'penalty': ['l1', 'l2'], 'C': loguniform(0.001, 100), 'solver': ['newton-cg','sag','saga','lbfgs']}

In [4]:
param, accuracy, precision, recall, f1 = [], [], [], [], []

# best oversampling: random_oversampling
X_train_oversampled, y_train_oversampled = random_oversampler(X_train, y_train)

# tuning
logreg = LogisticRegression()
logreg_clf = RandomizedSearchCV(logreg, logreg_distribution, n_iter=200, scoring=['accuracy', 'recall_weighted', 'precision_weighted', 'f1_weighted'], refit='f1_weighted', random_state=42)
logreg_search = logreg_clf.fit(X_train_oversampled, y_train_oversampled)

# update columns
param.append(logreg_search.cv_results_['params'])
accuracy.append(logreg_search.cv_results_['mean_test_accuracy'])
precision.append(logreg_search.cv_results_['mean_test_precision_weighted'])
recall.append(logreg_search.cv_results_['mean_test_recall_weighted'])
f1.append(logreg_search.cv_results_['mean_test_f1_weighted'])

In [5]:
results = pd.DataFrame({
    'model': "Logistic Regression",
    'parameters': np.array(param).flatten(),
    'accuracy': np.array(accuracy).flatten(),
    'precision': np.array(precision).flatten(),
    'recall': np.array(recall).flatten(),
    'f1': np.array(f1).flatten(),
}).sort_values(by="f1", ascending=False).reset_index(drop=True)
results.head(10)

,model,parameters,accuracy,precision,recall,f1
0,Logistic Regression,"{'C': 0.0029662981696401054, 'penalty': 'l1', ...",0.709570,0.772377,0.709570,0.703076
1,Logistic Regression,"{'C': 0.003190729641440915, 'penalty': 'l2', '...",0.701091,0.721646,0.701091,0.699298
2,Logistic Regression,"{'C': 0.0018130450933931477, 'penalty': 'l2', ...",0.697244,0.729251,0.697244,0.697533
3,Logistic Regression,"{'C': 0.00396251783257234, 'penalty': 'l2', 's...",0.699402,0.717271,0.699402,0.697061
4,Logistic Regression,"{'C': 0.003974905761171866, 'penalty': 'l2', '...",0.698920,0.716500,0.698920,0.696490
5,Logistic Regression,"{'C': 0.001180232156325819, 'penalty': 'l2', '...",0.695079,0.734790,0.695079,0.696412
6,Logistic Regression,"{'C': 0.0010600050132100263, 'penalty': 'l2', ...",0.694600,0.739180,0.694600,0.696189
7,Logistic Regression,"{'C': 0.026558434508499876, 'penalty': 'l1', '...",0.707837,0.706184,0.707837,0.696106
8,Logistic Regression,"{'C': 0.006429002142212086, 'penalty': 'l2', '...",0.702056,0.710120,0.702056,0.696026
9,Logistic Regression,"{'C': 0.0011947666257325973, 'penalty': 'l2', ...",0.694597,0.734243,0.694597,0.695889


Best Model

In [6]:
best_params = results.iloc[0]['parameters']
best_model = LogisticRegression(**best_params)
best_model.fit(X_train_oversampled, y_train_oversampled)
best_model

LogisticRegression(C=0.0029662981696401054, penalty='l1', solver='saga')

In [7]:
def print_results(actual, predictions):
    print(f"accuracy: {accuracy_score(actual, predictions)}")
    print(f"precision: {precision_score(actual, predictions, average='weighted')}")
    print(f"recall: {recall_score(actual, predictions, average='weighted')}")
    print(f"f1: {f1_score(actual, predictions, average='weighted')}")
    print(f"confusion matrix:\n{confusion_matrix(actual, predictions)}")

Predict on test set

In [8]:
test_pred = best_model.predict(X_test)
print_results(y_test, test_pred)

accuracy: 0.7380952380952381
precision: 0.7405881890614715
recall: 0.7380952380952381
f1: 0.7350914761169521
confusion matrix:
[[ 11  10   0]
 [ 24 113   0]
 [  2   8   0]]


Predict on full data

In [9]:
X_full = pd.concat([X_train, X_test])
y_full = pd.concat([y_train, y_test])

full_pred = best_model.predict(X_full)
print_results(y_full, full_pred)

accuracy: 0.8386897404202719
precision: 0.927023248627437
recall: 0.8386897404202719
f1: 0.873451165159429
confusion matrix:
[[  43   17    0]
 [ 187 1301   32]
 [   5   20   13]]


Business Metric

In [10]:
whole_df_x = pd.concat([X_train, X_test])
whole_df_x.shape

(1618, 11)

In [11]:
whole_df_x['year'] = whole_df_x.index.year

In [12]:
business_metric_results = pd.DataFrame(columns=['exp_strategy_annual_return', 'exp_benchmark_annual_return'])

for year in [2016, 2017, 2018, 2019, 2020, 2021, 2022]:
    year_data = whole_df_x[whole_df_x['year'] == year]
    year_data = year_data.drop(['year'], axis = 1)
    predict_x = best_model.predict(np.array(year_data))
    predictions = predict_x
    df_pred = pd.DataFrame({'prediction':predictions}, index=year_data.index)
    dates = df_pred.index

    if year == 2022:
        end_date = "2022-09-02"
    else:
        end_date = str(year+1) + "-01-01"
    df_prices = yf.download("^GSPC", start=dates[0], end=end_date)[['Adj Close']]
    df_prices = df_prices.reindex(df_pred.index)

    # create positions column
    positions = []
    prev = 0
    for i in range(len(df_pred)):
        if df_pred.iloc[i]['prediction'] == 0:
            positions.append(prev)
        else:
            prev = df_pred.iloc[i]['prediction']
            positions.append(prev)

    df_business = pd.DataFrame()
    df_business['stock_daily_log_return'] = np.log(df_prices /df_prices.shift(1))['Adj Close']
    df_business['prediction'] = df_pred['prediction']
    df_business['position'] = positions
    df_business['benchmark'] = 1 # long and hold strategy
    df_business["strategy_Returns"] = df_business["stock_daily_log_return"] * df_business["position"].shift(1)
    df_business["benchmark_Returns"] = df_business["stock_daily_log_return"] * df_business["benchmark"].shift(1)

    # Annual Mean Returns or Expected returns
    expected_strategy_annual_return = np.exp(df_business['strategy_Returns'].mean() * 252) - 1 
    expected_benchmark_annual_return = np.exp(df_business['benchmark_Returns'].mean() * 252) - 1 
    business_metric_results.loc[year] = [expected_strategy_annual_return, expected_benchmark_annual_return]

business_metric_results

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,exp_strategy_annual_return,exp_benchmark_annual_return
2016,-0.100999,0.107616
2017,-0.152486,0.185753
2018,0.131198,-0.070634
2019,-0.223869,0.288443
2020,0.041631,0.152929
2021,-0.206091,0.289230
2022,0.279334,-0.249185
